In [1]:
using Pkg
Pkg.instantiate()
Pkg.precompile()

Precompiling project...


In [2]:
using Interpolations
using QuadGK
using OrdinaryDiffEq

In [3]:
struct CosmoPar{T}
    Ωm::T
    Ωb::T
    h::T
    n_s::T
    σ8::T
end

struct Cosmology
    cosmo::CosmoPar
    # Power spectrum
    ks::Array
    pk0::Array
    dlogk
    lplk::AbstractInterpolation
    # Redshift and background
    zs::Array
    chis::Array
    chi::AbstractInterpolation
    z_of_chi::AbstractInterpolation
    chi_max
    Dzs::Array
    Dz::AbstractInterpolation
end

Cosmology(cpar::CosmoPar; nk=256, nz=256, tk_mode="BBKS") = begin
    # Compute linear power spectrum at z=0.
    ks = 10 .^ range(-4., stop=2., length=nk)
    dlogk = log(ks[2]/ks[1])
    if tk_mode== "Eis_Hu"
        tk = Tk_Eis_Hu(cpar, ks)
    elseif tk_mode== "BBKS"
        tk = TkBBKS(cpar, ks)
    else
        print("Transfer function not implemented")
    end
    pk0 = @. ks^cpar.n_s * tk
    σ8_2_here = _σR2(ks, pk0, dlogk, 8.0/cpar.h)
    norm = cpar.σ8^2 / σ8_2_here
    pk0[:] = pk0 .* norm
    # OPT: interpolation method
    pki = LinearInterpolation(log.(ks), log.(pk0))

    # Compute redshift-distance relation
    zs = range(0., stop=3., length=nz)
    norm = CLIGHT_HMPC / cpar.h
    chis = [quadgk(z -> 1.0/_Ez(cpar, z), 0.0, zz, rtol=1E-5)[1] * norm
            for zz in zs]
    # OPT: tolerances, interpolation method
    chii = LinearInterpolation(zs, chis)
    zi = LinearInterpolation(chis, zs)

    # ODE solution for growth factor
    z_ini = 1000.0
    a_ini = 1.0/(1.0+z_ini)
    ez_ini = _Ez(cpar, z_ini)
    d0 = [a_ini^3*ez_ini, a_ini]
    a_s = reverse(@. 1.0 / (1.0 + zs))
    prob = ODEProblem(_dgrowth!, d0, (a_ini, 1.0), cpar)
    sol = solve(prob, Tsit5(), reltol=1E-6,
                abstol=1E-8, saveat=a_s)
    # OPT: interpolation (see below), ODE method, tolerances
    # Note that sol already includes some kind of interpolation,
    # so it may be possible to optimize this by just using
    # sol directly.
    s = vcat(sol.u'...)
    Dzs = reverse(s[:, 2] / s[end, 2])
    # OPT: interpolation method
    Dzi = LinearInterpolation(zs, Dzs)

    Cosmology(cpar, ks, pk0, dlogk, pki,
              collect(zs), chis, chii, zi, chis[end],
              Dzs, Dzi)
end

Cosmology(Ωc, Ωb, h, n_s, σ8; nk=256, nz=256) = begin
    Ωm = Ωc + Ωb
    cpar = CosmoPar(Ωm, Ωb, h, n_s, σ8)

    Cosmology(cpar, nk=nk, nz=nz)
end

Cosmology() = Cosmology(0.25, 0.05, 0.67, 0.96, 0.81)

Cosmology

In [11]:
function get_zeq(cosmo::CosmoPar)
    wm=cosmo.Ωm*cosmo.h^2
    tfac=2.725 / 2.7
    return 2.5*10^4*wm*tfac^(-4)
end

function get_keq(cosmo::CosmoPar)
    wm=cosmo.Ωm*cosmo.h^2
    tfac=2.725/2.7
    return 7.46*10^(-2)*wm*tfac^(-2)
end

function get_zdrag(cosmo::CosmoPar)
    wb=cosmo.Ωb*cosmo.h^2
    wm=cosmo.Ωm*cosmo.h^2
    b1 = 0.313*wm^-0.419*(1+0.607*wm*0.674)
    b2 = 0.238*wm^0.223
    return 1291*((wm^0.251)/(1+0.659*wm^0.028))*(1+b1*wb^b2)
end

function R(cosmo::CosmoPar, z)
    wb=cosmo.Ωb*cosmo.h^2
    tfac=2.725 / 2.7
    return 31.5*wb*tfac^-4*(z/10^3)^-1
end


function get_rs(cosmo::CosmoPar)
    keq = get_keq(cosmo)
    zeq = get_zeq(cosmo)
    zd = get_zdrag(cosmo)
    rs = sqrt(1+R(cosmo, zd))+sqrt(R(cosmo, zd)+R(cosmo, zeq))
    rs /= 1+sqrt(R(cosmo, zeq))
    rs = log(rs)
    rs *= 2/(3*keq)*sqrt(6/R(cosmo, zeq))
    return rs
end
    
function G(y)
    return y*(-6*sqrt(1+y)+(2+3y)*log((sqrt(1+y)+1)/(sqrt(1+y)-1)))
end
    
function T0(cosmo::CosmoPar, k, ac, bc)
    tfac = 2.725 / 2.7
    wm=cosmo.Ωm*cosmo.h^2
    keq = get_keq(cosmo)
    q = k/(13.41*keq)
    C = 14.2/ac + 386/(1+69.9*q.^1.08)
    return (log(ℯ+1.8*bc*q))/(log(ℯ+1.8*bc*q)+C*q.^2)
end 
    
function Tc(cosmo::CosmoPar, k)
   Wc = cosmo.Ωm-cosmo.Ωb
   s = get_rs(cosmo)
   keq = get_keq(cosmo)
   tfac = 2.725 / 2.7
   wm=cosmo.Ωm*cosmo.h^2
   q = k/(13.41*keq)
   a1 = (46.9*wm)^0.670*(1+(32.1*wm)^-0.532)
   a2 = (12.0*wm)^0.424*(1+(45.0*wm)^-0.582)
   ac = (a1^(-cosmo.Ωb/cosmo.Ωm))*(a2^(-(cosmo.Ωb/cosmo.Ωm)^3))
   b1 = 0.944*(1+(458*wm)^(-0.708))^(-1)
   b2 = (0.395*wm)^(-0.0266)
   bc = 1/(1+b1*((Wc/cosmo.Ωm)^b2-1))
   f = 1/(1+(k*s/5.4)^4)
   return f*T0(cosmo, k, 1, bc) + (1-f)*T0(cosmo, k, ac, bc)
end 

function Tb(cosmo::CosmoPar, k)
   wm=cosmo.Ωm*cosmo.h^2
   wb=cosmo.Ωb*cosmo.h^2
   s = get_rs(cosmo)
   zd = get_zdrag(cosmo)
   zeq = get_zeq(cosmo)
   keq = get_keq(cosmo)
   ksilk = 1.6*wb^0.52*wm^0.73*(1+(10.4*wm)^-0.95)
   ab = 2.07*keq*s*(1+R(cosmo, zd))^(-3/4)*G((1+zeq)/(1+zd))
   bb =  0.5+(cosmo.Ωb/cosmo.Ωm)+(3-2*cosmo.Ωb/cosmo.Ωm)sqrt((17.2*wm)^2+1)
   bnode = 8.41*(wm)^0.435
   ss = s/(1+(bnode/(k*s))^3)^(1/3)
   Tb = (T0(cosmo, k, 1, 1)/(1+(k*s/5.2)^2))+(ab/(1+(bb/(k*s))^3))*exp(-1.4*(k/ksilk)^1.4)
   return Tb*(sin(k*ss)/(k*ss))
end 

function TkEisHu(cosmo::CosmoPar, k)
    Wc = cosmo.Ωm-cosmo.Ωb
    return (cosmo.Ωb/cosmo.Ωm)*Tb(cosmo, k)+(Wc/cosmo.Ωm)*Tc(cosmo, k)
end

growth_factor(cosmo::Cosmology, z) = cosmo.Dz(z)
function power_spectrum(cosmo::Cosmology, k, z)
    Dz2 = growth_factor(cosmo, z)^2
    @. exp(cosmo.lplk(log(k)))*Dz2
end

power_spectrum (generic function with 1 method)

In [12]:
cosmopar = CosmoPar(0.25, 0.05, 0.67, 0.96, 0.81)
#Cosmology = Cosmology(cosmopar, tk_mode="Eis_Hu") 

CosmoPar{Float64}(0.25, 0.05, 0.67, 0.96, 0.81)

In [6]:
Tc(cosmopar, [1.0])

MethodError: [91mMethodError: no method matching ^(::Array{Float64,1}, ::Int64)[39m
[91m[0mClosest candidates are:[39m
[91m[0m  ^([91m::Float16[39m, ::Integer) at math.jl:885[39m
[91m[0m  ^([91m::Regex[39m, ::Integer) at regex.jl:712[39m
[91m[0m  ^([91m::Missing[39m, ::Integer) at missing.jl:155[39m
[91m[0m  ...[39m

In [13]:
T0(cosmopar, [1.0], 1, 1)

MethodError: [91mMethodError: no method matching +(::Int64, ::Array{Float64,1})[39m
[91m[0mClosest candidates are:[39m
[91m[0m  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:529[39m
[91m[0m  +([91m::ChainRulesCore.NotImplemented[39m, ::Any) at /home/jaimerz/.julia/packages/ChainRulesCore/BYuIz/src/differential_arithmetic.jl:25[39m
[91m[0m  +([91m::ChainRulesCore.ZeroTangent[39m, ::Any) at /home/jaimerz/.julia/packages/ChainRulesCore/BYuIz/src/differential_arithmetic.jl:115[39m
[91m[0m  ...[39m

In [8]:
Tb(cosmopar, [1.0])

MethodError: [91mMethodError: convert(::Type{Union{}}, ::LinearAlgebra.Transpose{Float64,Array{Float64,1}}) is ambiguous. Candidates:[39m
[91m  convert(::Type{Union{}}, x) in Base at essentials.jl:169[39m
[91m  convert(::Type{T}, a::AbstractArray) where T<:Array in Base at array.jl:533[39m
[91m  convert(::Type{T}, arg) where T<:VecElement in Base at baseext.jl:8[39m
[91m  convert(T::Type{#s69} where #s69<:BitArray, a::AbstractArray) in Base at bitarray.jl:569[39m
[91m  convert(T::Type{#s664} where #s664<:LinearAlgebra.Bidiagonal, m::AbstractArray{T,2} where T) in LinearAlgebra at /usr/share/julia/stdlib/v1.4/LinearAlgebra/src/bidiag.jl:183[39m
[91m  convert(T::Type{#s664} where #s664<:SparseArrays.AbstractSparseMatrixCSC, m::AbstractArray{T,2} where T) in SparseArrays at /usr/share/julia/stdlib/v1.4/SparseArrays/src/sparsematrix.jl:561[39m
[91m  convert(::Type{T}, M::AbstractArray) where T<:OffsetArrays.OffsetArray in OffsetArrays at /home/jaimerz/.julia/packages/OffsetA

In [9]:
TkEisHu(cosmopar, [1.0])

MethodError: [91mMethodError: convert(::Type{Union{}}, ::LinearAlgebra.Transpose{Float64,Array{Float64,1}}) is ambiguous. Candidates:[39m
[91m  convert(::Type{Union{}}, x) in Base at essentials.jl:169[39m
[91m  convert(::Type{T}, a::AbstractArray) where T<:Array in Base at array.jl:533[39m
[91m  convert(::Type{T}, arg) where T<:VecElement in Base at baseext.jl:8[39m
[91m  convert(T::Type{#s69} where #s69<:BitArray, a::AbstractArray) in Base at bitarray.jl:569[39m
[91m  convert(T::Type{#s664} where #s664<:LinearAlgebra.Bidiagonal, m::AbstractArray{T,2} where T) in LinearAlgebra at /usr/share/julia/stdlib/v1.4/LinearAlgebra/src/bidiag.jl:183[39m
[91m  convert(T::Type{#s664} where #s664<:SparseArrays.AbstractSparseMatrixCSC, m::AbstractArray{T,2} where T) in SparseArrays at /usr/share/julia/stdlib/v1.4/SparseArrays/src/sparsematrix.jl:561[39m
[91m  convert(::Type{T}, M::AbstractArray) where T<:OffsetArrays.OffsetArray in OffsetArrays at /home/jaimerz/.julia/packages/OffsetA